# Алгоритм SIPP и его модификации

### Описание проекта

В этом проекте будет рассматриваться задача планирования траектории на сетчатых 4-connected графах с динамическими препятствиями. Целью проекта является реализация и/или сравнение алгоритмов:
- **AStar with timesteps**
- **SIPP**
- **Sub-optimal SIPP (WSIPP) with Re-expanded**
- **Sub-optimal SIPP (WSIPP) with Dublicate States**
- **Naive Anytime SIPP**

Все алгоритмы будут использовать манхэттеновскую эвристикy.

Алгоритм SIPP реализован без перераскрытий, так как доказано, что с такой эвристикой его решение будет оптимальным и без использования перераскрытий. Он будет сравниваться с AStar with timesteps.

Было решено, что наиболее интересной исследовательской задачей будет сравнение двух вариантов Sub-optimal SIPP.

Naive Anytime SIPP реализован с помощью последовательных запусков Sub-optimal SIPP с уменьшением параметра *w*. Было решено его реализовать, хотя он не является серьезным алгоритмом, так как он использует Sub-optimal SIPP.

Для начала импортируем все необходимые пакеты, вспомогательные функции и структуры и реализации алгоритмов:

In [1]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import time

from heapq import heappop, heappush, heapify
from random import randint
from IPython.display import HTML
from PIL import Image, ImageDraw, ImageOps
from IPython.display import Image as Img
from IPython.display import display
from sys import float_info
from datetime import datetime
from tqdm import tqdm

EPS = float_info.epsilon
%matplotlib notebook

In [2]:
from src.grid import Map, SafeMap, manhattan_distance
from src.utils import make_path, draw

In [3]:
from src.algo.astar_timesteps import astar_timesteps, SearchTree as SearchTreeAStarTimesteps
from src.algo.sipp import sipp, SearchTree as SearchTreeSIPP
from src.algo.wsipp_r import wsipp_r, SearchTree as SearchTreeWSIPPR
from src.algo.wsipp_d import wsipp_d, SearchTree as SearchTreeWSIPPD
from src.algo.naive_arsipp import naive_arsipp

### Описание хода эксперимента

Хотелось сравнить AStar with timesteps и SIPP, а также Sub-optimal SIPP (WSIPP) with Re-expanded и Sub-optimal SIPP (WSIPP) with Dublicate States.

Основной задачей было тестирование на картах с динамическими препятствиями, поэтому для сбора статистики были выбраны 4 карты: одна маленькая и 3 большие, обладающие разными свойствами.

Для каждой карты была вручную выбрана конфигурация расположения старта и цели.

Далее для каждой карты были случайным образом (с воспроизводимостью) сгенерированы 5 для маленькой и 50 для каждой большой карты конфигураций динамических препятствий. Количество динамических препятствий у конфигурации линейно зависит от её порядкового номера. 

Препятствия генерируются следующим образом: длина траектории (от 3 до 12), стартовая клетка пути и переходы на соседние клетки в таком количестве, чтобы соответствовало длине траектории. По стартовой клетке и переходам формируется список позиций траектории. Далее к нему в конец добавляется этот же список, но без первой и последней позиции и в обратном направлении. Когда препятствие достигает конца списка позиций, оно продолжает с начала списка, что формирует замкнутую траекторию, повторяющую себя в обоих направлениях.

Далее собирается статистика для каждого запуска на карте с конкретной конфигурацией динамических препятствий.

### Описание используемых карт

При сборе статистики используются 4 карты (можно посмотреть их изображения ниже): 
- **small**: маленькая карта-лабиринт
-  часть **32room_007**: лабиринт комнат с узкими проходами между свободными пространствами.
- часть **Paris_1_512**: город с достатоным объемом свободного пространства
- часть **random512-15-0**: рандомная карта-шум с большим количеством статических препятствий

<table><tr>
<td> <img src="./images/small.jpg" alt="32room_007" style="width: 400px;"/> </td>
<td> <img src="./images/32room_007.png" alt="32room_007" style="width: 200px;"/> </td>
<td> <img src="./images/Paris_1_512.png" alt="Paris_1_512" style="width: 200px;"/> </td>
<td> <img src="./images/random512_15_0.png" alt="random512_15_0" style="width: 200px;"/> </td>
</tr></table>


Карты отличаются общей идеей построения, внешним видом и плотностью расположения препятствий.

Карта **small** маленькая, простая и наглядная. На ней может быть удобнее отслеживать простые зависимости.

На карте **32room_007** узкие проходы соединяют свободные пространства. Это может давать интересные результаты при различных расположениях препятствий.

На карте **Paris_1_512** много широких проходов. Ожидается, что препятствия будут не слишком сильно мешать построению пути.

Карта **random512-15-0** довольно сильно загружена статическими препятствиями, что делает интересным добавление ещё и динамических препятствий.

### Описание собираемых статистик

В собираемую во время каждого запуска статистику входят:
- найден ли путь
- длина найденного пути
- во сколько раз найденный путь длиннее оптимального (для WSIPP)
- количество шагов алгоритма (это также количество раскрытий)
- количество созданных *Node*
- время работы

Перечисленные показатели дают информацию о:
- оптимальности решений алгоритма
- времени работы (количество шагов и фактическое) 
- использованной памяти

Этих параметров должно хватать для оценки и сравнения предложенных для исследования алгоритмов.

In [4]:
from src.launch import launch_astar_timesteps, launch_sipp, launch_wsipp

In [5]:
astar_timesteps_small = launch_astar_timesteps("small", 5, manhattan_distance, SearchTreeAStarTimesteps)
astar_timesteps_room = launch_astar_timesteps("32room_007", 50, manhattan_distance, SearchTreeAStarTimesteps)
astar_timesteps_paris = launch_astar_timesteps("Paris_1_512", 50, manhattan_distance, SearchTreeAStarTimesteps)
astar_timesteps_ran = launch_astar_timesteps("random512-15-0", 50, manhattan_distance, SearchTreeAStarTimesteps)


2it [00:00, 12.45it/s]

Path found! Length: 52. Nodes created: 7311. Number of steps: 1692. Time: 0:00:00.053659
Path found! Length: 52. Nodes created: 7206. Number of steps: 1669. Time: 0:00:00.095369
Path found! Length: 52. Nodes created: 6964. Number of steps: 1635. Time: 0:00:00.054419


5it [00:00, 11.40it/s]

Path found! Length: 54. Nodes created: 9022. Number of steps: 2128. Time: 0:00:00.069146
Path found! Length: 52. Nodes created: 6768. Number of steps: 1604. Time: 0:00:00.060605


KeyboardInterrupt: 

In [22]:
sipp_small = launch_sipp("small", 5, manhattan_distance, SearchTreeSIPP)
sipp_room = launch_sipp("32room_007", 50, manhattan_distance, SearchTreeSIPP)
sipp_paris = launch_sipp("Paris_1_512", 50, manhattan_distance, SearchTreeSIPP)
sipp_ran = launch_sipp("random512-15-0", 50, manhattan_distance, SearchTreeSIPP)


3it [00:00, 23.86it/s]

Path found! Length: 52. Nodes created: 780. Number of steps: 241. Time: 0:00:00.011120
Path found! Length: 53. Nodes created: 4021. Number of steps: 247. Time: 0:00:00.050236
Path found! Length: 52. Nodes created: 4738. Number of steps: 253. Time: 0:00:00.044206
Path found! Length: 52. Nodes created: 6545. Number of steps: 256. Time: 0:00:00.068327


5it [00:00, 13.24it/s]

Path found! Length: 53. Nodes created: 12362. Number of steps: 288. Time: 0:00:00.138350



2it [00:00, 18.81it/s]

...

Path found! Length: 110. Nodes created: 34736. Number of steps: 1469. Time: 0:00:00.346824


8it [00:04,  1.56it/s]

Path found! Length: 110. Nodes created: 17431. Number of steps: 797. Time: 0:00:00.176710


8it [00:05,  1.49it/s]


KeyboardInterrupt: 

In [19]:
wsippr_small = launch_wsipp("small", wsipp_r, 5, 1.4, manhattan_distance, SearchTreeWSIPPR)
wsippr_room = launch_wsipp("32room_007", wsipp_r, 50, 1.4, manhattan_distance, SearchTreeWSIPPR)
wsippr_paris = launch_wsipp("Paris_1_512", wsipp_r, 50, 1.4, manhattan_distance, SearchTreeWSIPPR)
wsippr_ran = launch_wsipp("random512-15-0", wsipp_r, 50, 1.4, manhattan_distance, SearchTreeWSIPPR)


3it [00:00, 25.85it/s]

Path found! Length: 52. Coefficient: 1.0. Nodes created: 692. Number of steps: 216. Time: 0:00:00.008644
Path found! Length: 52. Coefficient: 1.0. Nodes created: 1293. Number of steps: 225. Time: 0:00:00.013215
Path found! Length: 70. Coefficient: 1.0. Nodes created: 2870. Number of steps: 297. Time: 0:00:00.022659
Path found! Length: 54. Coefficient: 1.0. Nodes created: 6255. Number of steps: 253. Time: 0:00:00.061977


5it [00:00, 10.04it/s]

Path found! Length: 53. Coefficient: 1.0. Nodes created: 12450. Number of steps: 327. Time: 0:00:00.115569



0it [00:00, ?it/s]

...

Path found! Length: 116. Coefficient: 1.0545454545454545. Nodes created: 26696. Number of steps: 435. Time: 0:00:00.292499


17it [00:16,  1.54s/it]

Path found! Length: 115. Coefficient: 1.0454545454545454. Nodes created: 11313. Number of steps: 448. Time: 0:00:00.115022


17it [00:18,  1.08s/it]


KeyboardInterrupt: 

In [1]:
labels = ["A*", "WA* w=1.05", "WA* w=1.15", "WA* w=1.5", "WA* w=2", "WA* w=3", "WA* w=5", "WA* w=10"]
labels_small = ["A*", "1.05", "1.15", "1.5", "2", "3", "5", "10"]


def label_algo(i):
    return labels[i]
    

def color_algo(i):
    if i == 0:
        return "red"
    if i == 1:
        return "yellow"
    if i == 2:
        return "green"
    if i == 3:
        return "blue"
    if i == 4:
        return "grey"
    if i == 5:
        return "black"
    if i == 6:
        return "purple"
    if i == 7:
        return "pink"


In [ ]:
def coef_grafics():
    
    xmin = 0
    xmax = 120
    ymin = 0.8
    ymax = 5

    x = np.linspace(xmin, xmax, 10000)
    
    
    for i, stat in enumerate(all_stat_32room_007):
        plt.plot(list(range(len(stat["coef"]))), stat["coef"], label=label_algo(i), color=color_algo(i))
        
        plt.grid(ls=':')
        plt.xlabel('Сложность задания'), plt.ylabel('Коэффициент длины пути относительно оптимального')
        plt.legend(loc = 'upper left')
          
    plt.title ("32room_007") 
    plt.show()
    
    
    for i, stat in enumerate(all_stat_ht_bartrand_n):
        plt.plot(list(range(len(stat["coef"]))), stat["coef"], label=label_algo(i), color=color_algo(i))
        
        plt.grid(ls=':')
        plt.xlabel('Сложность задания'), plt.ylabel('Коэффициент длины пути относительно оптимального')
        plt.legend(loc = 'upper left')
          
    plt.title ("ht_bartrand_n") 
    plt.show()
    
    
    for i, stat in enumerate(all_stat_Paris_1_512):
        plt.plot(list(range(len(stat["coef"]))), stat["coef"], label=label_algo(i), color=color_algo(i))
        
        plt.grid(ls=':')
        plt.xlabel('Сложность задания'), plt.ylabel('Коэффициент длины пути относительно оптимального')
        plt.legend(loc = 'upper left')
          
    plt.title ("Paris_1_512") 
    plt.show()
    
    
    for i, stat in enumerate(all_stat_random512_15_0):
        plt.plot(list(range(len(stat["coef"]))), stat["coef"], label=label_algo(i), color=color_algo(i))
        
        plt.grid(ls=':')
        plt.xlabel('Сложность задания'), plt.ylabel('Коэффициент длины пути относительно оптимального')
        plt.legend(loc = 'upper left')
          
    plt.title ("random512_15_0") 
    plt.show()
    
    
coef_grafics()    

In [ ]:
def runtime_grafics():
    
    xmin = 0
    xmax = 120
    ymin = 0.8
    ymax = 10

    x = np.linspace(xmin, xmax, 10000)
    
    
    for i, stat in enumerate(all_stat_32room_007):
        plt.plot(list(range(len(stat["time"]))), list(map(lambda t: t.total_seconds(), stat["time"])), label=label_algo(i), color=color_algo(i))
        
        plt.grid(ls=':')
        plt.xlabel('Сложность задания'), plt.ylabel('Время работы')
        plt.legend(loc = 'upper left')
          
    plt.title ("32room_007") 
    plt.show()
    
    
    for i, stat in enumerate(all_stat_ht_bartrand_n):
        plt.plot(list(range(len(stat["time"]))), list(map(lambda t: t.total_seconds(), stat["time"])), label=label_algo(i), color=color_algo(i))
        
        plt.grid(ls=':')
        plt.xlabel('Сложность задания'), plt.ylabel('Время работы')
        plt.legend(loc = 'upper left')
          
    plt.title ("ht_bartrand_n") 
    plt.show()
    
    
    for i, stat in enumerate(all_stat_Paris_1_512):
        plt.plot(list(range(len(stat["time"]))), list(map(lambda t: t.total_seconds(), stat["time"])), label=label_algo(i), color=color_algo(i))
        
        plt.grid(ls=':')
        plt.xlabel('Сложность задания'), plt.ylabel('Время работы')
        plt.legend(loc = 'upper left')
          
    plt.title ("Paris_1_512") 
    plt.show()
    
    
    for i, stat in enumerate(all_stat_random512_15_0):
        plt.plot(list(range(len(stat["time"]))), list(map(lambda t: t.total_seconds(), stat["time"])), label=label_algo(i), color=color_algo(i))
        
        plt.grid(ls=':')
        plt.xlabel('Сложность задания'), plt.ylabel('Время работы')
        plt.legend(loc = 'upper left')
          
    plt.title ("random512_15_0") 
    plt.show()
    
    
runtime_grafics()    

In [ ]:
def runtime_boxes():
    
    plt.figure(figsize=(10, 10))
    
    plt.subplot(2, 2, 1)
    means = []
    for i, stat in enumerate(all_stat_32room_007):
        stat_runtime_norm = []
        for i, time in enumerate(stat["time"]):
            stat_runtime_norm.append(time.total_seconds() / all_stat_32room_007[0]["time"][i].total_seconds())
        
        means.append(np.array(stat_runtime_norm))
     
    plt.boxplot(means, labels=labels_small)    
    plt.title ("32room_007", fontsize=10) 
   

    plt.subplot(2, 2, 2)
    means = []
    for i, stat in enumerate(all_stat_ht_bartrand_n):
        stat_runtime_norm = []
        for i, time in enumerate(stat["time"]):
            stat_runtime_norm.append(time.total_seconds() / all_stat_ht_bartrand_n[0]["time"][i].total_seconds())
        
        means.append(np.array(stat_runtime_norm))
     
    plt.boxplot(means, labels=labels_small)    
    plt.title ("ht_bartrand_n", fontsize=10) 
    
    
    plt.subplot(2, 2, 3)
    means = []
    for i, stat in enumerate(all_stat_Paris_1_512):
        stat_runtime_norm = []
        for i, time in enumerate(stat["time"]):
            stat_runtime_norm.append(time.total_seconds() / all_stat_Paris_1_512[0]["time"][i].total_seconds())
        
        means.append(np.array(stat_runtime_norm))
     
    plt.boxplot(means, labels=labels_small)    
    plt.title ("Paris_1_512", fontsize=10) 
     
    
    plt.subplot(2, 2, 4)
    means = []
    for i, stat in enumerate(all_stat_random512_15_0):
        stat_runtime_norm = []
        for i, time in enumerate(stat["time"]):
            stat_runtime_norm.append(time.total_seconds() / all_stat_random512_15_0[0]["time"][i].total_seconds())
        
        means.append(np.array(stat_runtime_norm))
     
    plt.boxplot(means, labels=labels_small)    
    plt.title ("random512_15_0", fontsize=10) 
     
    
    plt.subplots_adjust(wspace=0.2, hspace=0.4)
    plt.show()
    
runtime_boxes()    